## Import libraries

In [1]:
import os
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import geopandas as gpd
from plotly.subplots import make_subplots
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LinearRegression
from scipy.stats import chi2_contingency

pd.set_option('display.max_columns', None)

## 2. Read the training data

In [2]:
def get_data(name, base_dir=None):
    # If base_dir is not provided, use the current working directory
    if base_dir is None:
        base_dir = os.getcwd()
    
    # Construct the path to the Data directory
    data_dir = os.path.join(base_dir, '..', 'Data')
    
    # Construct the full file path
    file_name = f"{name}.csv"
    file_path = os.path.join(data_dir, file_name)
    
    # Check if the file exists
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_name} does not exist in the specified path.")
    
    # Read and return the CSV file
    return pd.read_csv(file_path)

In [3]:
train = get_data('eda_data')
# for visulaization we are using only 1000 rows not entire dataset because of computation limitations
train = train.sample(n=1000, random_state=42).reset_index(drop=True)
# load json file
with open("../GeoJson/germany-states2.geojson", "r") as file:
    geojson_data = json.load(file)
    
# view of the dataset
print(f'shape of the dataframe: {train.shape}')
train.head(2)

shape of the dataframe: (1000, 30)


,serviceCharge,picturecount,pricetrend,telekomUploadSpeed,totalRent,baseRent,livingSpace,noRooms,residents,zip_area_sq_km,stateName,heatingType,telekomTvOffer,newlyConst,balcony,firingTypes,hasKitchen,cellar,condition,interiorQual,petsAllowed,lift,typeOfFlat,geo_plz,garden,date,cityCode,cityName,yearConstructed_category,floor_category
0,105.68,8,1.59,40,624.18,442.54,66.05,2,19352,6.266675,Sachsen_Anhalt,district_heating,ONE_YEAR_FREE,False,True,district_heating:local_heating,False,True,unknown,normal,negotiable,True,apartment,39104,False,Feb 2020,39,Magdeburg,1973-1997,floor_6_45
1,94.00,4,3.33,40,459.00,317.00,49.44,2,32776,21.502596,Nordrhein_Westfalen,district_heating,ONE_YEAR_FREE,False,False,gas,False,True,unknown,unknown,unknown,False,ground_floor,45665,False,Sep 2018,45,Essen,1952-1973,ground_floor_and_below


## 3. Dashboard charts

### Dashboard State wise analysis

In [ ]:
# totalprice vs stateName
avg_rent_by_state = train.groupby('stateName')['totalRent'].mean().sort_values(ascending= False).reset_index()

# Create the box plot
fig = px.bar(avg_rent_by_state, x='stateName', y='totalRent', width = 900 ,height= 500)
fig.update_layout(title='Average Total Rent per Month by State',
                   xaxis_title='State Name',
                   yaxis_title='Average Total Rent')

# customize text appearance
fig.update_traces(texttemplate='€%{y:.2f}', textposition='outside')
fig.show()

In [ ]:
# change in rent price over time
avg_rent_by_state = (train
                     .groupby(['stateName','date'])['totalRent']
                     .mean()
                     .reset_index()
                    ).sort_values(['date'], ascending=False)

# Create the box plot
fig = px.line(avg_rent_by_state, x='date', y='totalRent', color = 'stateName',width = 800 ,height= 500)
fig.update_layout(title='Average Total Rent per Month by State',
                   xaxis_title='Date',
                    yaxis_title='Average Total Monthly Rent (€)',
                    legend_title='State Name')
fig.show()

In [ ]:
# Avg total rent per month by state
result = train.groupby(['typeOfFlat','stateName'])['totalRent'].mean().unstack()

# Create figure
fig = go.Figure()

# Add traces for each state
for state in result.columns:
    fig.add_trace(go.Bar(
        x=result.index,
        y=result[state],
        name=state
    ))

# Update layout
fig.update_layout(
    title='Average Total Rent by Flat Type and State',
    xaxis_title='Type of Flat',
    yaxis_title='Average Total Rent',
    barmode='group',
    legend_title='State',
    height=600,
    width=800
)

# Show the plot
fig.show()

In [ ]:
# Calculate average price per state
temp_df = train.groupby('stateName')['totalRent'].mean().reset_index()
temp_df['stateName'] = temp_df['stateName'].str.replace('_', '-')

# Load GeoJSON file for German states
with open('../GeoJSON/germany-states2.geojson') as f:
    geojson_data = json.load(f)  
# Verify if all state names match the GeoJSON
geojson_states = [feature['properties']['name'] for feature in geojson_data['features']]
missing_states = set(temp_df['stateName']) - set(geojson_states)

if missing_states:
    print(f"Missing states in GeoJSON: {missing_states}")
else:
    print("All states match the GeoJSON.")


# Create the choropleth map
fig = px.choropleth(
    temp_df,
    geojson=geojson_data,
    locations='stateName',          
    featureidkey='properties.name', 
    color='totalRent',              
    # hover_name='stateName',         
    color_continuous_scale= 'Viridis_r',  
)

# Update layout for better appearance
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(
    title='Average Total Rent per State in Germany',
    coloraxis_colorbar=dict(title='Average Rent (€)'),
    margin={"r":0,"t":50,"l":0,"b":0}  
)

# Show the figure
fig.show()

### Dashboard 2  city analysis

In [ ]:
# Avg rent per city in Germany
avg_rent_by_city = train.groupby('cityName')['totalRent'].mean().sort_values(ascending=False).head(10).reset_index()

# Create the bar plot
fig = px.bar(avg_rent_by_city, x='cityName', y='totalRent', width = 1000 ,height= 500)
fig.update_layout(title='Average Total Rent per Month by Top 10 Cities',
                   xaxis_title='City Name',
                   yaxis_title='Average Total Monthly Rent ')

# customize text appearance
fig.update_traces(texttemplate='€%{y:.2f}', textposition='outside')
fig.show()


In [ ]:

# Load GeoJSON file for 2-digit postal codes
geojson_path = '../GeoJSON/plz-2stellig.geojson'
with open(geojson_path, 'r') as f:
    geojson_data = json.load(f)

# Extract postal codes from the GeoJSON
geojson_plz = [str(feature['properties']['plz']) for feature in geojson_data['features']]

# Ensure cityCode in train is formatted as strings with leading zeros if needed
train['cityCode'] = train['cityCode'].astype(str).str.zfill(2)

# Check for missing postal codes
missing_plz = set(train['cityCode']) - set(geojson_plz)
if missing_plz:
    print(f"Missing PLZ in GeoJSON: {missing_plz}")
else:
    print("All postal codes match the GeoJSON.")

# Validate totalRent column
if train['totalRent'].isna().any():
    print("Warning: 'totalRent' contains NaN values.")
train['totalRent'] = pd.to_numeric(train['totalRent'], errors='coerce')

# Create the choropleth map
fig = px.choropleth(
    train,
    geojson=geojson_data,
    locations='cityCode',           
    featureidkey='properties.plz',  
    color='totalRent',              
    hover_name='cityName',          
    hover_data={'totalRent': ':.2f'}, 
    color_continuous_scale='Viridis_r',  
    labels={'totalRent': 'Average Rent (€)'} ) 

# Update layout for better appearance
fig.update_geos(
    fitbounds="locations",
    visible=False
)

fig.update_layout(
    title={
        'text': 'Average Total Rent per 2-Digit Postal Code in Germany',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    coloraxis_colorbar=dict(
        title='Average Rent (€)',
        title_side='right'
    ),
    margin={"r": 0, "t": 50, "l": 0, "b": 0},
    paper_bgcolor='white',
    plot_bgcolor='white'
)

# Enable hover interactivity
fig.update_layout(
    hovermode='closest',  
    dragmode=False        
)

# Display the figure
fig.show()

### general charts

In [ ]:

temp_df = train[['serviceCharge', 'date']].sort_values('date', ascending=False)
temp_df = temp_df[temp_df['serviceCharge'] <= 6000]

# Check if 'date' column exists and convert it to ordinal for regression
temp_df['date_ordinal'] = pd.to_datetime(temp_df['date']).map(pd.Timestamp.toordinal)

# Fit linear regression model
X = temp_df['date_ordinal'].values.reshape(-1, 1)
y = temp_df['serviceCharge'].values
model = LinearRegression()
model.fit(X, y)

# Get the slope and intercept
slope = model.coef_[0]
intercept = model.intercept_

# Create the box plot using Plotly
fig = px.box(temp_df, x='date', y='serviceCharge', width=600, height=400)

# Predict values for the trend line
predicted_service_charge = model.predict(X)

# Add trend line to the figure
fig.add_trace(go.Scatter(x=temp_df['date'], y=predicted_service_charge, mode='lines',
                         name='Trend Line', line=dict(color='red')))

# Add annotation for the trend line equation
equation_text = f"Trendline: y = {slope:.2f}x + {intercept:.2f}"
fig.add_annotation(
    x=temp_df['date'].iloc[-1],  # Position at the last date
    y=predicted_service_charge[-1],  # Position at the last predicted value
    text=equation_text,
    showarrow=True,
    arrowhead=2,
    ax=0,
    ay=-40,
    font=dict(size=12),
    bgcolor="white",
)

# Update layout and appearance
fig.update_layout(
    title='Average Service Charge per Year with Trend Line',
    xaxis_title='Date',
    yaxis_title='Avg Service Charge (€)'
)

# Customize y-axis appearance
fig.update_yaxes(tickprefix='€')

# Show the figure
fig.show()

In [ ]:
# Create the box plot
fig = px.box(train, x='noRooms', y='totalRent', width=800, height=400)

# Update layout
fig.update_layout(
    title='Total Rent Distribution by Number of Rooms',
    xaxis_title='Number of Rooms',
    yaxis_title='Total Rent (€)',
)

fig.update_traces(marker=dict(size=2))  # Adjust the size of outlier points
# Show the figure
fig.show()

## Dashboard

In [4]:
import json
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import html, dcc, Input, Output
import plotly.io as pio
import pandas as pd

# import data
train = pd.read_csv('../Data/eda_data.csv')
train = train.sample(n=1000, random_state=42).reset_index(drop=True)

# Load GeoJSON file for German states
with open('../GeoJSON/germany-states2.geojson') as f:
    geojson_data = json.load(f)

# Set default Plotly theme
pio.templates.default = "plotly_white"

# State-level data preparation
avg_rent_by_state_bar = train.groupby('stateName')['totalRent'].mean().reset_index()
avg_rent_by_state_bar = avg_rent_by_state_bar.sort_values('totalRent', ascending=False)

avg_rent_by_state_line = (
    train.groupby(['stateName', 'date'])['totalRent']
    .mean()
    .reset_index()
    .sort_values(['date'], ascending=False)
)

# City-level data preparation
avg_rent_by_city_bar = train.groupby('cityName')[['totalRent', 'baseRent', 'serviceCharge']].mean().reset_index()
avg_rent_by_city_bar = avg_rent_by_city_bar.sort_values('totalRent', ascending=False)

avg_rent_by_city_line = (
    train.groupby(['cityName', 'date'])['totalRent']
    .mean()
    .reset_index()
    .sort_values(['date'], ascending=False)
)

# Create Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Germany Rent Analysis Dashboard", style={'textAlign': 'center'}),
    
    dcc.Tabs([
        # State Tab
        dcc.Tab(label='State Analysis', children=[
            html.Div([
                dcc.Dropdown(
                    id='state-dropdown',
                    options=[{'label': 'All States', 'value': 'all'}] +
                            [{'label': state, 'value': state} for state in train['stateName'].unique()],
                    value='all',
                    placeholder="Select a state",
                    style={'width': '50%', 'margin': '20px auto'}
                ),
                
                # Add metric cards for state analysis
                html.Div(id='state-metrics', className='metric-container'),
                
                html.Div([
                    html.Div([
                        dcc.Graph(id='state-rent-distribution'),
                    ], style={'width': '50%'}),
                    
                    html.Div([
                        dcc.Graph(id='state-bar-chart'),
                    ], style={'width': '50%'}),
                ], style={'display': 'flex'}),

                html.Div([
                    html.Div([
                        dcc.Graph(id='state-line-chart'),
                    ], style={'width': '50%'}),

                    html.Div([
                        dcc.Graph(id='state-heatmap'),
                    ], style={'width': '50%'}),
                ], style={'display': 'flex', 'marginTop': '20px'}),
            ]),
        ]),
        
        # City Tab
        dcc.Tab(label='City Analysis', children=[
            html.Div([
                dcc.Dropdown(
                    id='city-dropdown',
                    options=[{'label': 'All Cities', 'value': 'all'}] +
                            [{'label': city, 'value': city} for city in train['cityName'].unique()],
                    value='all',
                    placeholder="Select a city",
                    style={'width': '50%', 'margin': '20px auto'}
                ),
                
                # Add metric cards for city analysis
                html.Div(id='city-metrics', className='metric-container'),
                
                html.Div([
                    html.Div([
                        dcc.Graph(id='city-price-components'),
                    ], style={'width': '50%'}),
                    
                    html.Div([
                        dcc.Graph(id='city-features-impact'),
                    ], style={'width': '50%'}),
                ], style={'display': 'flex'}),

                html.Div([
                    html.Div([
                        dcc.Graph(id='city-year-analysis'),
                    ], style={'width': '50%'}),

                    html.Div([
                        dcc.Graph(id='city-property-types'),
                    ], style={'width': '50%'}),
                ], style={'display': 'flex', 'marginTop': '20px'}),
            ]),
        ]),
    ]),
])

@app.callback(
    [Output('state-rent-distribution', 'figure'),
     Output('state-bar-chart', 'figure'),
     Output('state-line-chart', 'figure'),
     Output('state-heatmap', 'figure'),
     Output('state-metrics', 'children')],
    [Input('state-dropdown', 'value')]
)
def update_state_charts(selected_state):
    df = train.copy()
    if selected_state != 'all':
        df = df[df['stateName'] == selected_state]
    
    # Rent Distribution
    dist_fig = px.box(
        df,
        x='stateName',
        y='totalRent',
        title='Rent Distribution by State'
    )
    
    # Bar Chart - Average Rents with highlighting
    bar_fig = px.bar(
        avg_rent_by_state_bar,
        x='stateName',
        y='totalRent',
        title='Average Total Rent by State'
    )
    
    # Update bar colors and opacity based on selection
    if selected_state != 'all':
        colors = ['rgba(31, 119, 180, 0.3)' if state != selected_state else 'rgb(31, 119, 180)' 
                 for state in avg_rent_by_state_bar['stateName']]
        bar_fig.update_traces(marker_color=colors)
    
    bar_fig.update_traces(
        texttemplate='€%{y:.2f}',
        textposition='outside'
    )
    
    # Line Chart - Trends
    line_fig = px.line(
        df.groupby(['date', 'stateName'])['totalRent'].mean().reset_index(),
        x='date',
        y='totalRent',
        color='stateName',
        title='Rent Trends Over Time'
    )
    
    # Heatmap - Correlations
    numeric_cols = ['totalRent', 'baseRent', 'serviceCharge', 'livingSpace', 'noRooms']
    corr_matrix = df[numeric_cols].corr()
    heatmap_fig = px.imshow(
        corr_matrix,
        title='Correlation Heatmap',
        labels=dict(color="Correlation")
    )
    
    # Metrics
    metrics = html.Div([
        html.Div([
            html.H4(f"Average Total Rent: €{df['totalRent'].mean():.2f}"),
            html.H4(f"Average Living Space: {df['livingSpace'].mean():.1f} m²"),
            html.H4(f"Average Rooms: {df['noRooms'].mean():.1f}"),
        ], style={'textAlign': 'center'})
    ])
    
    return dist_fig, bar_fig, line_fig, heatmap_fig, metrics

@app.callback(
    [Output('city-price-components', 'figure'),
     Output('city-features-impact', 'figure'),
     Output('city-year-analysis', 'figure'),
     Output('city-property-types', 'figure'),
     Output('city-metrics', 'children')],
    [Input('city-dropdown', 'value')]
)
def update_city_charts(selected_city):
    df = train.copy()
    if selected_city != 'all':
        df = df[df['cityName'] == selected_city]
    
    # Price Components
    components_fig = px.bar(
        avg_rent_by_city_bar.head(15),
        x='cityName',
        y=['baseRent', 'serviceCharge'],
        title='Rent Components by City (Top 15)',
        barmode='stack'
    )
    components_fig.update_layout(xaxis_tickangle=-45)
    
    # Features Impact
    features = ['balcony', 'hasKitchen', 'cellar', 'garden', 'lift']
    impact_data = []
    for feature in features:
        avg_with = df[df[feature] == 1]['totalRent'].mean()
        avg_without = df[df[feature] == 0]['totalRent'].mean()
        impact_data.append({
            'feature': feature,
            'with_feature': avg_with,
            'without_feature': avg_without
        })
    
    impact_df = pd.DataFrame(impact_data)
    features_fig = px.bar(
        impact_df,
        x='feature',
        y=['with_feature', 'without_feature'],
        title='Impact of Features on Rent',
        barmode='group',
        labels={'value': 'Average Rent (€)', 'variable': 'Feature Presence'}
    )
    
    # Year Analysis
    year_fig = px.box(
        df,
        x='yearConstructed_category',
        y='totalRent',
        title='Rent Distribution by Construction Year'
    )
    
    # Property Types
    type_fig = px.bar(
        df.groupby('typeOfFlat')['totalRent'].mean().reset_index(),
        x='typeOfFlat',
        y='totalRent',
        title='Average Rent by Property Type'
    )
    type_fig.update_traces(texttemplate='€%{y:.2f}', textposition='outside')
    
    # Metrics
    metrics = html.Div([
        html.Div([
            html.H4(f"Average Total Rent: €{df['totalRent'].mean():.2f}"),
            html.H4(f"Average Service Charge: €{df['serviceCharge'].mean():.2f}"),
            html.H4(f"Average Living Space: {df['livingSpace'].mean():.1f} m²"),
        ], style={'textAlign': 'center'})
    ])

    return components_fig, features_fig, year_fig, type_fig, metrics

In [5]:
import socket
h_name = socket.gethostname()
IP_address = socket.gethostbyname(h_name)

print(f"Dashboard is running at http://{IP_address}:8050")

Dashboard is running at http://134.155.227.178:8050


In [6]:
app.run_server(host=IP_address, port=8050)